# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [57]:
import logging
import os
import csv

import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException


from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.52.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [58]:
ws = Workspace.from_config()
experiment_name = 'project3'

experiment=Experiment(ws, experiment_name)

cluster_name = "udacity"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           min_node_count = 1,
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, timeout_in_minutes = 30)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [59]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required

subscription_id = '7c03dd83-6b95-43b1-9f53-23dfd07e8803'
resource_group = 'AZP-102-Temp_AI-RG'
workspace_name = 'AZP-102_Temp_AI_ML_POC'

workspace = Workspace(subscription_id, resource_group, workspace_name)
ds = workspace.get_default_datastore()
#data_ref = ds.path('UI/2023-08-09_061836_UTC/CO2 Emissions.csv').as_mount()
input_data = Dataset.File.from_files(ds.path('UI/2023-08-09_061836_UTC/CO2 Emissions.csv')).as_named_input('input').as_mount()


In [4]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1, evaluation_interval=2) 

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    '--max_depth': choice(1, 5, 10, 20),
    '--min_samples_leaf': choice(1, 2, 4)
})

if "training" not in os.listdir():
    os.mkdir("./training")

if "outputs" not in os.listdir():
    output_dir = os.mkdir("./outputs")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='../conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                                   script='train_hyperdrive.py',
                                   compute_target=cluster_name,
                                   environment=sklearn_env, 
                                   arguments=['--data', input_data])

hyperdrive_run_config = HyperDriveConfig(run_config=src,
                                    hyperparameter_sampling=param_sampling,
                                    policy=early_termination_policy,
                                    primary_metric_name='r2_score',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=64,
                                    max_concurrent_runs=4,
                                    )

In [5]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_8b56455c-9481-459c-92d7-e142323e9cef
Web View: https://ml.azure.com/runs/HD_8b56455c-9481-459c-92d7-e142323e9cef?wsid=/subscriptions/7c03dd83-6b95-43b1-9f53-23dfd07e8803/resourcegroups/azp-102-temp_ai-rg/workspaces/AZP-102_Temp_AI_ML_POC&tid=eb70b763-b6d7-4486-8555-8831709a784e

Streaming azureml-logs/hyperdrive.txt

[2023-08-28T11:37:20.418604][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-08-28T11:37:20.9347681Z][SCHEDULER][INFO]Scheduling job, id='HD_8b56455c-9481-459c-92d7-e142323e9cef_0' 
[2023-08-28T11:37:21.0869431Z][SCHEDULER][INFO]Scheduling job, id='HD_8b56455c-9481-459c-92d7-e142323e9cef_1' 
[2023-08-28T11:37:21.2716036Z][SCHEDULER][INFO]Scheduling job, id='HD_8b56455c-9481-459c-92d7-e142323e9cef_3' 
[2023-08-28T11:37:21.2734346Z][SCHEDULER][INFO]Scheduling job, id='HD_8b56455c-9481-459c-92d7-e142323e9cef_2' 
[2023-08-28T11:37:21.218250][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution targe

{'runId': 'HD_8b56455c-9481-459c-92d7-e142323e9cef',
 'target': 'udacity',
 'status': 'Completed',
 'startTimeUtc': '2023-08-28T11:37:19.5173Z',
 'endTimeUtc': '2023-08-28T11:46:25.413606Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"r2_score","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'cb63388e-581b-4334-8a33-89a1a944d99a',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1035-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.52.0',
  'space_size': '12',
  'score': '0.9618546245466579',
  'best_child_run_id': 'HD_8b56455c-9481-459c-92d7-e142323e9cef_4',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_8b56455c-9481-459c-92d7-e142323e9cef_4'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

No such comm: efcdb7dc0faf4b848ee61fdf1a8feb3e
No such comm: efcdb7dc0faf4b848ee61fdf1a8feb3e
No such comm: efcdb7dc0faf4b848ee61fdf1a8feb3e
No such comm: efcdb7dc0faf4b848ee61fdf1a8feb3e
No such comm: efcdb7dc0faf4b848ee61fdf1a8feb3e
No such comm: efcdb7dc0faf4b848ee61fdf1a8feb3e
No such comm: efcdb7dc0faf4b848ee61fdf1a8feb3e
No such comm: efcdb7dc0faf4b848ee61fdf1a8feb3e
No such comm: efcdb7dc0faf4b848ee61fdf1a8feb3e
No such comm: efcdb7dc0faf4b848ee61fdf1a8feb3e
No such comm: efcdb7dc0faf4b848ee61fdf1a8feb3e
No such comm: efcdb7dc0faf4b848ee61fdf1a8feb3e
No such comm: efcdb7dc0faf4b848ee61fdf1a8feb3e
No such comm: efcdb7dc0faf4b848ee61fdf1a8feb3e
No such comm: efcdb7dc0faf4b848ee61fdf1a8feb3e
No such comm: efcdb7dc0faf4b848ee61fdf1a8feb3e
No such comm: efcdb7dc0faf4b848ee61fdf1a8feb3e
No such comm: efcdb7dc0faf4b848ee61fdf1a8feb3e
No such comm: efcdb7dc0faf4b848ee61fdf1a8feb3e
No such comm: efcdb7dc0faf4b848ee61fdf1a8feb3e
No such comm: efcdb7dc0faf4b848ee61fdf1a8feb3e
No such comm:

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [7]:
#TODO: Save the best model
best_run = hyperdrive_run.get_best_run_by_primary_metric()

# Retrieve the best model from the best run
best_model = best_run.register_model(model_name='best_hyperdrive_model', model_path='outputs/model.pkl')


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [125]:
from azureml.core import Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment

# Load the registered model
model = Model(workspace, 'best_hyperdrive_model')

# Create an inference configuration
inference_config = InferenceConfig(entry_script='endpoint_score.py', environment=Environment.from_conda_specification(name='inference-env', file_path='../conda_dependencies.yml'))

# Deploy the model to ACI
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
service = Model.deploy(workspace, 'hyperdrive-model', [model], inference_config=inference_config, deployment_config=deployment_config, overwrite=True)

# Wait for the deployment to complete
service.wait_for_deployment(show_output=True)
print(service.get_logs())

azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-08-29 13:59:12+00:00 Creating Container Registry if not exists.
2023-08-29 13:59:12+00:00 Registering the environment.
2023-08-29 13:59:12+00:00 Use the existing image.
2023-08-29 13:59:13+00:00 Submitting deployment to compute.
2023-08-29 13:59:16+00:00 Checking the status of deployment hyperdrive-model..
2023-08-29 14:01:59+00:00 Checking the status of inference endpoint hyperdrive-model.
Succeeded
ACI service creation operation finished, operation "Succeeded"
2023-08-29T14:01:35,425956533+00:00 - rsyslog/run 
2023-08-29T14:01:35,432903870+00:00 - gunicorn/run 
2023-08-29T14:01:35,438999402+00:00 | gunicorn/run | 
2023-08-29T14:01:35,448237952+00:00 | gunicorn/run | ###############################################
2023-08-29T14:01:35,454301984+00:00 | gunicorn/run | AzureML Container Runtime Inf

Before the ACI Webservice was deploayed a local webservice was used to test and debugg the rest endpoint

In [124]:
from azureml.core.webservice import LocalWebservice
import json
model = Model(workspace, 'best_hyperdrive_model')
inference_config = InferenceConfig(entry_script='endpoint_score.py', environment=Environment.from_conda_specification(name='inference-env', file_path='../conda_dependencies.yml'))
deployment_config = LocalWebservice.deploy_configuration(port=9001)

service = Model.deploy(ws, 'local-service-hyperdrivemodel', [model], inference_config, deployment_config=deployment_config)

data = {'data': [ {"Engine Size(L)":2.0,"Cylinders":4.0,"Fuel Consumption City (L\\/100 km)":9.9,"Fuel Consumption Hwy (L\\/100 km)":8.3}]}
json_data = json.dumps(data)
print(service.run(input_data=json_data))

service.reload()
print(service.run(input_data=json_data))

azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'


Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 63601acb5e4343dc852586ce80dd0c51.azurecr.io
Logging into Docker registry 63601acb5e4343dc852586ce80dd0c51.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 63601acb5e4343dc852586ce80dd0c51.azurecr.io/azureml/azureml_8b3f26c7c34b2bcee5b939f3b8755b66
 ---> a9dd09bbf375
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 8dbd3db91107
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjdjMDNkZDgzLTZiOTUtNDNiMS05ZjUzLTIzZGZkMDdlODgwMyIsInJlc291cmNlR3JvdXBOYW1lIjoiYXpwLTEwMi10ZW1wX2FpLXJnIiwiYWNjb3VudE5hbWUiOiJhenAtMTAyX3RlbXBfYWlfbWxfcG9jIiwid29ya3NwYWNlSWQiOiI2MzYwMWFjYi01ZTQzLTQzZGMtODUyNS04NmNlODBkZDBjNTEifSwibW9kZWxzIjp7fSwibW9kZWxzSW5mbyI6e319 | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 548c24105913
 ---> 8eb4690c66ce
Step 4/5 : RUN mv '/var/azureml-app/tmpl_4emfn7.py' /var/azureml-app/main.py
 ---> Ru

TODO: In the cell below, send a request to the web service you deployed to test it.

In [126]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data = {"data":
        [
          {"Engine Size(L)":2.0,
          "Cylinders":4.0,
          "Fuel Consumption City (L\\/100 km)":9.9,
          "Fuel Consumption Hwy (L\\/100 km)":8.3
          }
      ]
    }

body = str.encode(json.dumps(data))

url = 'http://710c509e-7035-4f07-9a45-c025e84ddcb8.westeurope.azurecontainer.io/score'


headers = {'Content-Type':'application/json'}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'[212.0]'


Enable application insights

In [127]:
from azureml.core import Workspace
from azureml.core.webservice import Webservice

model_name='hyperdrive-model'

service = Webservice(name=model_name, workspace=ws)
service.update(enable_app_insights=True)

TODO: In the cell below, print the logs of the web service and delete the service

In [129]:
print(service.get_logs())

2023-08-29T14:15:25,341633657+00:00 - gunicorn/run 
2023-08-29T14:15:25,334319157+00:00 - rsyslog/run 
2023-08-29T14:15:25,356313957+00:00 | gunicorn/run | 
2023-08-29T14:15:25,360944357+00:00 | gunicorn/run | ###############################################
2023-08-29T14:15:25,364041557+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-08-29T14:15:25,369196057+00:00 - nginx/run 
2023-08-29T14:15:25,371358657+00:00 | gunicorn/run | ###############################################
2023-08-29T14:15:25,379977257+00:00 | gunicorn/run | 
2023-08-29T14:15:25,385177757+00:00 | gunicorn/run | 
2023-08-29T14:15:25,394849857+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230620.v4
2023-08-29T14:15:25,396639457+00:00 | gunicorn/run | 
2023-08-29T14:15:25,403856357+00:00 | gunicorn/run | 
2023-08-29T14:15:25,405296557+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml_7eec2c8971b9410f92147a7e257297e7/bin:/opt/mi